![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [2]:
import sqlite3

In [3]:
!ls -la

total 13472
drwxr-xr-x   9 philipklein-rodick  staff      288 Aug 23 13:57 .
drwxr-xr-x   4 philipklein-rodick  staff      128 Aug 26 11:54 ..
drwxr-xr-x   3 philipklein-rodick  staff       96 Aug 23 13:09 .ipynb_checkpoints
-rwxr-xr-x   1 philipklein-rodick  staff  1067008 Aug 23 13:05 Chinook_Sqlite.sqlite
-rwxr-xr-x   1 philipklein-rodick  staff  5415936 Aug 23 13:05 flights.db
drwxr-xr-x  13 philipklein-rodick  staff      416 Aug 23 13:05 img
-rwxr-xr-x   1 philipklein-rodick  staff    26243 Aug 23 13:05 sql-to-pandas-apm.ipynb
-rwxr-xr-x   1 philipklein-rodick  staff    58356 Aug 23 14:54 sql-to-pandas.ipynb
-rwxr-xr-x   1 philipklein-rodick  staff   317533 Aug 23 13:05 sql-to-pandas.slides.html


#### Load a DB object with `connect` and `cursor`

In [4]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [5]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [6]:
cursor.description

(('name', None, None, None, None, None, None),)

In [7]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [8]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [9]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [10]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [11]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [12]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [13]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [14]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [15]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

In [16]:
cursor.execute("""select "Flatiron School\"""").fetchall()

[('Flatiron School',)]

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [17]:
cursor.execute("""SELECT distinct country 
                  FROM airlines""").fetchall()

[(None,),
 ('United States',),
 ('South Africa',),
 ('United Kingdom',),
 ('Russia',),
 ('Thailand',),
 ('Canada',),
 ('Australia',),
 ('Singapore',),
 ('Belgium',),
 ('Mexico',),
 ('Spain',),
 ('France',),
 ('United Arab Emirates',),
 ('Republic of Korea',),
 ('Pakistan',),
 ('Libya',),
 ('Gambia',),
 ('Ivory Coast',),
 ('Ukraine',),
 ('Democratic Republic of the Congo',),
 ('Iran',),
 ('Finland',),
 ('Brazil',),
 ('Colombia',),
 ('AEROCENTER',),
 ('Ghana',),
 ('Kenya',),
 ('Liberia',),
 ('Togo',),
 ('Somali Republic',),
 ('Morocco',),
 ('Canadian Territories',),
 ('Dominican Republic',),
 ('Japan',),
 ('Albania',),
 ('Nigeria',),
 ('Germany',),
 ('Slovenia',),
 ('Czech Republic',),
 ('Benin',),
 ('AEROCESAR',),
 ('Greece',),
 ('Chile',),
 ('Tanzania',),
 ('Bolivia',),
 ('Italy',),
 ('Sweden',),
 ('Argentina',),
 ('Sierra Leone',),
 ('Indonesia',),
 ('Senegal',),
 ('Afghanistan',),
 ('Uganda',),
 ('Bosnia and Herzegovina',),
 ('Gabon',),
 ('Angola',),
 ('Uzbekistan',),
 ('Namibia',),


In [18]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [19]:
cursor.execute("""
SELECT *
FROM routes
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('airline', None, None, None, None, None, None)
('airline_id', None, None, None, None, None, None)
('source', None, None, None, None, None, None)
('source_id', None, None, None, None, None, None)
('dest', None, None, None, None, None, None)
('dest_id', None, None, None, None, None, None)
('codeshare', None, None, None, None, None, None)
('stops', None, None, None, None, None, None)
('equipment', None, None, None, None, None, None)


In [20]:
cursor.execute("""
SELECT *
FROM airports
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('id', None, None, None, None, None, None)
('name', None, None, None, None, None, None)
('city', None, None, None, None, None, None)
('country', None, None, None, None, None, None)
('code', None, None, None, None, None, None)
('icao', None, None, None, None, None, None)
('latitude', None, None, None, None, None, None)
('longitude', None, None, None, None, None, None)
('altitude', None, None, None, None, None, None)
('offset', None, None, None, None, None, None)
('dst', None, None, None, None, None, None)
('timezone', None, None, None, None, None, None)


In [21]:
cursor.execute("""SELECT a.latitude, a.longitude, r.airline, r.source, r.dest
                    FROM airports as a
                    JOIN routes as r on r.source_id = a.id  """).fetchone()


('-6.081689', '145.391881', 'CG', 'GKA', 'HGU')

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [22]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY count DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)


In [23]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY count DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)
('Spain', 142)
('France', 97)
('Germany', 94)
('South Africa', 81)
('Nigeria', 80)


In [24]:
# What about airports by timezones?
cursor.execute("""
SELECT timezone, count(*) AS count
FROM airports
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)
('America/Los_Angeles', 226)
('Asia/Chongqing', 222)
('Europe/London', 193)
('America/Toronto', 159)
('Asia/Calcutta', 141)


In [25]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [26]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [27]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
128,A796351,Max,08/24/2019 12:03:00 PM,08/24/2019 12:03:00 PM,05/31/2010,Transfer,Partner,Cat,Neutered Male,9 years,Domestic Shorthair,Black/White
157,A618405,Max,08/23/2019 06:38:00 PM,08/23/2019 06:38:00 PM,07/15/2010,Adoption,NaN,Dog,Neutered Male,9 years,American Pit Bull Terrier Mix,White/Blue
206,A653408,Max,08/23/2019 11:33:00 AM,08/23/2019 11:33:00 AM,05/06/2009,Return to Owner,NaN,Dog,Neutered Male,10 years,American Bulldog Mix,White/Black
352,A783700,Max,08/20/2019 06:36:00 PM,08/20/2019 06:36:00 PM,11/04/2017,Adoption,NaN,Dog,Neutered Male,1 year,Labrador Retriever Mix,Chocolate/White
1132,A744193,Max,08/08/2019 03:17:00 PM,08/08/2019 03:17:00 PM,09/27/2016,Adoption,Foster,Dog,Neutered Male,2 years,Cairn Terrier Mix,Tan/White
1178,A789119,Max,08/07/2019 06:56:00 PM,08/07/2019 06:56:00 PM,12/17/2014,Adoption,Foster,Dog,Neutered Male,4 years,Catahoula Mix,Red
1422,A801136,Max,08/04/2019 11:26:00 AM,08/04/2019 11:26:00 AM,03/22/2008,Rto-Adopt,NaN,Dog,Intact Male,11 years,Miniature Pinscher,Red
1598,A800872,Max,08/01/2019 04:37:00 PM,08/01/2019 04:37:00 PM,06/12/2019,Adoption,NaN,Dog,Neutered Male,1 month,German Shepherd Mix,Tan/Black
1756,A800791,Max,07/29/2019 06:07:00 PM,07/29/2019 06:07:00 PM,07/26/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Pit Bull,Brown/White
1891,A788916,Max,07/27/2019 05:16:00 PM,07/27/2019 05:16:00 PM,11/26/2018,Return to Owner,NaN,Dog,Neutered Male,7 months,Pit Bull Mix,Brown Brindle/White


In [28]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
1,A802752,NaN,08/26/2019 01:17:00 PM,08/26/2019 01:17:00 PM,08/22/2016,Transfer,Partner,Dog,Intact Male,3 years,Shih Tzu,Black/White
2,A802979,Percy,08/26/2019 01:00:00 PM,08/26/2019 01:00:00 PM,08/26/2004,Return to Owner,NaN,Dog,Spayed Female,15 years,Labrador Retriever Mix,Tan
3,A802978,Seeker,08/26/2019 12:43:00 PM,08/26/2019 12:43:00 PM,08/25/2013,Return to Owner,NaN,Dog,Intact Female,6 years,Chihuahua Shorthair,Black/Tan
4,A791541,*Nabisco,08/26/2019 12:34:00 PM,08/26/2019 12:34:00 PM,09/27/2017,Transfer,Partner,Dog,Neutered Male,1 year,Pit Bull Mix,Red/White
5,A802826,Lovely,08/26/2019 12:14:00 PM,08/26/2019 12:14:00 PM,01/23/2017,Return to Owner,NaN,Dog,Spayed Female,2 years,Chihuahua Shorthair,Black/Brown


## Goal 6: Transfering from sqlite to pandas

In [29]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [30]:
# Which countries have the highest amount of active airports?
df = pd.read_sql_query("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY 2 DESC
LIMIT 10
""", pd_con)
df

,country,count
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34
5,Australia,26
6,China,25
7,Spain,24
8,Brazil,23
9,France,22


## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [31]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [32]:
# How many tables are in the database?
cur.execute("SELECT count(*) as table_count FROM sqlite_master WHERE type = 'table';").fetchone()

(11,)

In [41]:
# What's the primary key of each table?
table_names = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()
for table_name in table_names:
    tname = table_name[0]
    # table_name is still a tuple of length 1, so we have to unpack to get the table name value
    results = cur.execute(f"PRAGMA table_info({tname});").fetchall()

    # column name is 2nd column,
    # primary key is the 4th column, according to https://www.sqlite.org/pragma.html#pragma_table_info
    pk_column_name = [x[1] for x in results ]
    print(f"{tname}.{pk_column_name[0]}")

Album.AlbumId
Artist.ArtistId
Customer.CustomerId
Employee.EmployeeId
Genre.GenreId
Invoice.InvoiceId
InvoiceLine.InvoiceLineId
MediaType.MediaTypeId
Playlist.PlaylistId
PlaylistTrack.PlaylistId
Track.TrackId


In [46]:
# What foreign keys join the tables together?
table_names = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()
for table_name in table_names:
    tname = table_name[0]
    # table_name is still a tuple of length 1, so we have to unpack to get the table name value
    results = cur.execute(f"PRAGMA foreign_key_list({tname});").fetchall()
    if results:
        print(tname, 'table:')
        print(*results, sep='\n')

Album table:
(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')
Customer table:
(0, 0, 'Employee', 'SupportRepId', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')
Employee table:
(0, 0, 'Employee', 'ReportsTo', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')
Invoice table:
(0, 0, 'Customer', 'CustomerId', 'CustomerId', 'NO ACTION', 'NO ACTION', 'NONE')
InvoiceLine table:
(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Invoice', 'InvoiceId', 'InvoiceId', 'NO ACTION', 'NO ACTION', 'NONE')
PlaylistTrack table:
(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Playlist', 'PlaylistId', 'PlaylistId', 'NO ACTION', 'NO ACTION', 'NONE')
Track table:
(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE')
(1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE')
(2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')


In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
SELECT max(InvoiceDate) AS max_date,
min(InvoiceDate) AS min_date
FROM Invoice""").fetchall()

In [ ]:
# What tables would you need to answer "what is your most popular track?"

# Track (for track info)
# InvoiceLine (for track puchase quantity)

In [ ]:
# What values from each table?

# Track.TrackId, Track.Name, Track.AlbumId
# InvoiceLine.TrackId, InvoiceLine.Quantity
# Album.AlbumId, Album.ArtistId
# Artist.ArtistId, Artist.Name

In [ ]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
SELECT
t.name as track_name,
sum(il.quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

In [ ]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
SELECT
t.name as track_name,
a.name as artist_name,
sum(il.quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
JOIN Album USING (AlbumId)
JOIN Artist a USING (ArtistId)
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

## Reflection